In [ ]:
import requests
import pandas as pd
from urllib.parse import quote
from dotenv import load_dotenv
import os


load_dotenv()  # 현재 프로젝트 폴더(또는 지정한 경로)에 있는 .env 파일 로드
# 네이버 개발자 센터에서 발급받은 값으로 변경하세요
client_id = os.getenv('NAVER_CLIENT_ID')
client_secret = os.getenv('NAVER_CLIENT_SECRET')

# 검색어
search_query = "LLM"

# 수집할 페이지 수 (예: 10페이지 -> 총 100개)
num_pages = 10

# 결과를 담을 리스트
results = []

for page in range(1, num_pages+1):
    # 네이버 검색 API 문서에 따르면, start 파라미터는 1부터 시작하여
    # 10씩 증가(예: 1, 11, 21, 31, ...)
    start = (page - 1) * 10 + 1

    # 요청할 URL
    url = "https://openapi.naver.com/v1/search/blog"

    # 요청 헤더
    headers = {
        "X-Naver-Client-Id": client_id,
        "X-Naver-Client-Secret": client_secret
    }

    # 요청 파라미터
    params = {
        "query": quote(search_query),  # 검색어
        "display": 10,                 # 페이지당 문서 개수
        "start": start,               # 검색 시작 위치
        "sort": "sim"                 # 정렬 옵션("sim": 정확도순, "date": 날짜순)
    }

    # GET 요청
    response = requests.get(url, headers=headers, params=params)
    
    # 응답 결과 (JSON)
    data = response.json()
    
    # 검색 결과의 'items' 키 아래에 결과 목록이 존재
    items = data.get('items', [])

    # items에서 필요한 데이터 추출 후 리스트에 저장
    for item in items:
        results.append({
            "title": item["title"],
            "link": item["link"],
            "description": item["description"],
            "bloggername": item["bloggername"],
            "bloggerlink": item["bloggerlink"],
            "postdate": item["postdate"]
        })

# 판다스 데이터프레임으로 변환
df = pd.DataFrame(results)

# 결과 확인
df